In [1]:
from cryptocmd import CmcScraper
import pandas as kunfu
import numpy as dragon
import pylab as p
import matplotlib.pyplot as plot
from collections import Counter
import re

import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
# from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

print('All good!')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
scraper = CmcScraper("ETH","04-08-2010", "03-08-2022")
xrp_json_data = scraper.get_data("json")
df = scraper.get_dataframe()[['Date', 'Close']]
df.head()

In [3]:
len(df)

2554

In [4]:
df.tail()

,Date,Close
2549,2015-08-11,1.067860
2550,2015-08-10,0.708448
2551,2015-08-09,0.701897
2552,2015-08-08,0.753325
2553,2015-08-07,2.772120


In [6]:
# Splitting into train and test
Date = df[['Date']]
Close = df[['Close']]
X_train, X_test, y_train, y_test = train_test_split(Date, Close, train_size = 0.78)

NameError: name 'train_test_split' is not defined

Exploratory Analysis

In [3]:
data = train['Close']
Date1 = train['Date']
train1 = train[['Date','Close']]

# Setting the Date as Index
train2 = train1.set_index('Date')
train2.sort_index(inplace=True)
print (type(train2))
print (train2.head())
plot.plot(train2)
plot.xlabel('Date', fontsize=12)
plot.ylabel('Price in USD', fontsize=12)
plot.title("Closing price distribution of bitcoin", fontsize=15)
plot.show()

NameError: name 'train' is not defined